In [2]:
import pyspark
from pyspark.sql.functions import *

In [13]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn import metrics
import numpy as np
# from sklearn.externals.six import StringIO 
from six import StringIO
from IPython.display import Image  
import pydotplus

In [2]:
import pandas as pd  # This is always assumed but is included here as an introduction.
import numpy as np
import matplotlib.pyplot as plt

In [3]:
! pip install matplotlib pandas scikit-learn pydotplus

In [11]:
! pip install tensorflow

In [6]:
! pip uninstall h5py 

Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Would remove:
    /opt/conda/lib/python3.9/site-packages/h5py-3.1.0.dist-info/*
    /opt/conda/lib/python3.9/site-packages/h5py.libs/libaec-9c9e97eb.so.0.0.10
    /opt/conda/lib/python3.9/site-packages/h5py.libs/libhdf5-00e8fae8.so.200.0.0
    /opt/conda/lib/python3.9/site-packages/h5py.libs/libhdf5_hl-383c339f.so.200.0.0
    /opt/conda/lib/python3.9/site-packages/h5py.libs/libsz-e7aa62f5.so.2.0.1
    /opt/conda/lib/python3.9/site-packages/h5py.libs/libz-eb09ad1d.so.1.2.3
    /opt/conda/lib/python3.9/site-packages/h5py/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [10]:
! conda install -y -c anaconda h5py

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - h5py


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.10.14 |                0         128 KB  anaconda
    ------------------------------------------------------------
                                           Total:         128 KB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2021.10.~ --> anaconda::ca-certificates-2020.10.14-0



ca-certificates-2020 | 128 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
